In [1]:
from selenium.webdriver import Chrome
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time

import warnings
warnings.filterwarnings(action='ignore')

from dateutil.relativedelta import relativedelta
import datetime as dt

In [23]:
# 미국 원유생산 원유시추 원유재고 석유재고 사우디 선물유가 
keywords = '러시아 중간유분 석유수요 이란 중국 베이커휴즈 비OPEC 생산량 산유국들 베네수엘라 이라크 트럼프 나이지리아 원유생산량'.split()
keywords

['러시아',
 '중간유분',
 '석유수요',
 '이란',
 '중국',
 '베이커휴즈',
 '비OPEC',
 '생산량',
 '산유국들',
 '베네수엘라',
 '이라크',
 '트럼프',
 '나이지리아',
 '원유생산량']

In [20]:
import os
os.listdir('./data/news/Korean/'+keyword)


FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: './data/news/Korean/naver'

In [ ]:
homepath = './data/news/Korean/'
for keyword in keywords:
    filepath = homepath+keyword
    try:
        os.listdir(filepath)
    except:
        os.mkdir(filepath)
        
    start_date = dt.datetime(2017,2,10)
    while start_date < dt.datetime(2022,2,10):
        date_lst = []
        title_lst = []
        summary_lst = []
        
        sdname = start_date.strftime('%Y%m%d')
        sdtxt = start_date.strftime('%Y.%m.%d')
        start_date += relativedelta(months=1)
        edname = start_date.strftime('%Y%m%d')
        edtxt = start_date.strftime('%Y.%m.%d')
        
        for page in range(1,401):
            # 전체
            # url = f'https://search.naver.com/search.naver?where=news&sm=tab_pge&query={keyword}&sort=2&photo=0&field=0&pd=3&ds={sdtxt}&de={edtxt}&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from{sdname}to{edname},a:all&start=41&page={page}'
            # 지면기사
            url = f'https://search.naver.com/search.naver?where=news&query={keyword}&sm=tab_opt&sort=2&photo=3&field=0&pd=3&ds={sdtxt}&de={edtxt}&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom{sdname}to{edname}&is_sug_officeid=0&page={page}'
            soup = BeautifulSoup(requests.get(url).text,'html.parser')
            for article in soup.select('#main_pack > section > div > div.group_news > ul.list_news > li'):
                title = article.select('div > div.news_area > a')[0].text
                if len(article.select('div > div.news_area > div.news_info > div.info_group > span')) == 1:
                    date = article.select('div > div.news_area > div.news_info > div.info_group > span')[0].text
                else:
                    date = article.select('div > div.news_area > div.news_info > div.info_group > span')[1].text
                try:
                    summary = article.select('div > div.news_area > div.news_dsc > div.dsc_wrap > a')[0].text
                except:
                    summary = np.nan
                date_lst.append(date)
                title_lst.append(title)
                summary_lst.append(summary)
        
        pd.DataFrame({'date':date_lst, 'title':title_lst, 'summary':summary_lst}).to_csv(filepath+f'/news_{keyword}_{sdname}_{edname}.csv')